In [18]:
import numpy as np
from keras.models import Model
from keras.layers import Input, BatchNormalization, Dense
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import PReLU, LeakyReLU

channels = 3
        
def pixel_shuffle(in_map, h, w, c = channels):
    out_map = in_map.reshape(batchsize, h, w, 2, 2, c)
    out_map = out_map.transpose(0, 3, 1, 4, 2, 5)
    out_map = out_map.reshape(batchsize, 2 * h, 2 * w, c)
    return out_map

def upsampling(in_map, h, w, c):
    out_map = Conv2D(filters = 4 * c, 
                     kernel_size = 3,
                     strides = 1,
                     padding = "same")(in_map)
    out_map = pixel_shuffle(out_map, h, w, c)
    out_map = PReLU()(out_map)
    return out_map

def residual_block(in_map):
    out_map = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(in_map)
    out_map = BatchNormalization()(out_map)
    out_map = PReLU()(out_map)
    out_map = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(in_map)
    out_map = BatchNormalization()(out_map)
    out_map = Add()([out_map, in_map])
    return out_map

def d_block(in_map, filters, kernel_size, strides, padding):
    d = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding)(in_map)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha = 0.2)(d)
    return d
    

    
        
        
        

In [19]:


class Generator():
    def __init__(self, L_h, L_w, c, r):
        self.L_h = L_h
        self.L_w = L_w
        self.c = c
        self.r = r
        self.H_h = self.L_h * self.r
        self.H_w = self.L_w * self.r
        
        
        
    def generate(self, input_img):
        input_img = Input(shape = (self.L_h, self.L_w, self.c))
        g = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same")(input_img)
        g = PReLU()(g)
        
        for _ in range(5):
            g = residual_block(g)
            
        g = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(g)
        g = BatchNormalization()(g)
        g = Add([g, input_img])
        
        n = self.r
        i = 1
        while(n % 2 == 0):
            g = upsampling(g, self.L_h * i, self.L_w * i, self.c)
            i = i * 2
            n = n // 2

        output_img = Conv2D(filters = 3, kernel_size = 9, strides = 1, padding = "same")(g)

        return output_img
    
    
    
class Discriminator():
    def __init__(self, h, w, c):
        self.h = h
        self.w = w
        self.c = c
    
    def discriminate(self, input_img):
        input_img = Input(shape = (self.h, self.w, self.c))
        d = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(input_img)
        d = d_block(d, filters = 64, kernel_size = 3, strides = 2, padding = "same")
        d = d_block(d, filters = 128, kernel_size = 3, strides = 1, padding = "same")
        d = d_block(d, filters = 128, kernel_size = 3, strides = 2, padding = "same")
        d = d_block(d, filters = 256, kernel_size = 3, strides = 1, padding = "same")
        d = d_block(d, filters = 256, kernel_size = 3, strides = 2, padding = "same")
        d = d_block(d, filters = 512, kernel_size = 3, strides = 1, padding = "same")
        d = d_block(d, filters = 512, kernel_size = 3, strides = 2, padding = "same")
        d = Dense(1024)(d)
        d = LeakyReLU(alpha = 0.2)(d)
        output = Dense(1, activation = "sigmoid")(d)
        
        return output
    
    
class SRGAN(Generator, Discriminator):
    pass
        









In [20]:
from PIL import Image
D = Discriminator(480, 640, 3)
img = Image.open("../../images/train/image_1.png")
img = np.array(img)
valid = D.discriminate(img)
print(valid)

Tensor("dense_2/Sigmoid:0", shape=(?, 30, 40, 1), dtype=float32)
